# Test Sync_UI functionality of PyZOS

In [1]:
import os
import pyzos.zos as zos
import warnings
warnings.simplefilter(action='ignore')

## Test zPushLens() 

Build a basic optical system and Push to the UI

In [2]:
# Create an OpticalSystem instance with sync_ui parameter set to True

osys = zos.OpticalSystem(sync_ui=True)

In [3]:
# Set aperture Aperture
sdata = osys.pSystemData
sdata.pAperture.pApertureValue = 40

# Set fields
field = sdata.pFields.AddField(0, 5.0, 1.0)

# Set wavelength
sdata.pWavelengths.SelectWavelengthPreset(zos.Const.WavelengthPreset_d_0p587);

In [4]:
# Create surfaces
osys.zInsertNewSurfaceAt(1)
osys.zInsertNewSurfaceAt(1)
osys.zSetSurfaceData(1, thick=10, material='N-BK7', comment='front of lens')
osys.zSetSurfaceData(2, thick=50, comment='rear of lens')
osys.zSetSurfaceData(3, thick=350, comment='Stop is free to move')

In [5]:
osys.zPushLens(update=1) # with 1, the lens data will be updated in the UI

In [6]:
# Set solves on surfaces
osys.pLDE.GetSurfaceAt(1).pRadiusCell.MakeSolveVariable()
osys.pLDE.GetSurfaceAt(1).pThicknessCell.MakeSolveVariable()
osys.pLDE.GetSurfaceAt(2).pRadiusCell.MakeSolveVariable()
osys.pLDE.GetSurfaceAt(2).pThicknessCell.MakeSolveVariable()
osys.pLDE.GetSurfaceAt(3).pThicknessCell.MakeSolveVariable()

True

In [7]:
osys.zSetDefaultMeritFunctionSEQ(ofType=0, ofData=1, ofRef=0, rings=2, 
                                 arms=0, grid=0, useGlass=True, glassMin=3, 
                                 glassMax=15, glassEdge=3, useAir=True, 
                                 airMin=0.5, airMax=1000, airEdge=0.5)

In [8]:
osys.zPushLens(1)

In [9]:
# Additional Operand
mfe = osys.pMFE
operand1 = mfe.InsertNewOperandAt(1)
operand1 = zos.wrapped_zos_object(operand1)
operand1.ChangeType(zos.Const.MeritOperandType_EFFL)
operand1.pTarget = 400.0
operand1.pWeight = 1.0

In [10]:
osys.zPushLens(1)

In [11]:
# Local optimization
local_opt = osys.pTools.OpenLocalOptimization()
local_opt.pAlgorithm = zos.Const.OptimizationAlgorithm_DampedLeastSquares
local_opt.pCycles = zos.Const.OptimizationCycles_Automatic
local_opt.pNumberOfCores = 8
local_opt.RunAndWaitForCompletion()
local_opt.Close()

True

In [12]:
osys.zPushLens(1)

In [13]:
# Save lens file (optional)
sdir = osys.pTheApplication.pSamplesDir
file_out = os.path.join(sdir, 'Sequential', 'Objectives', 
                        'Single Lens Example wizard+EFFL.zmx')
osys.SaveAs(file_out)

## Test zGetRefresh()

In [14]:
# LDE
lde = osys.pLDE

We will change the thickness of surface 2 in the UI.

In [15]:
# Get row 2
row2 = lde.GetRowAt(2)

In [16]:
thickness2 = zos.wrapped_zos_object(row2.GetCellAt(3)).pValue
float(thickness2)

94.10279977269181

Change the thickness value in the UI manually.

In [17]:
osys.zGetRefresh() # zGetRefresh to update the lens in the COM server

In [18]:
thickness2 = zos.wrapped_zos_object(row2.GetCellAt(3)).pValue
float(thickness2)

150.0

In [19]:
osys.New(False)

In [20]:
osys.zPushLens(1)